In [2]:
import pandas as pd
import numpy as np

matches = pd.read_html('https://www.calciocsi.it/2023-2024/VARESE/CSI-VARESE-OPEN-B/OPEN-B3/RISULTATI')

In [3]:
my_team = "ASD SAN MAURIZIO NERO"

locations = {
    'ASD ORATORI CARDANO OPEN A': 'Via G.Matteotti, 12, Cardano al Campo',
    'ASD SAN MAURIZIO BLU': 'Via San Pio X, 57, Cassano Magnago',
    'ASD SAN MAURIZIO NERO': 'Via L. Da Vinci, 8, Cassano Magnago',
    'CENTRO DELLA GIOVENTÙ': 'Via Don Minzoni, 7, Gallarate',
    'CSI NUOVA ABBIATE': 'Via Oslavia, 17, Tradate',
    'DUDE CAFÈ': 'Via Romagnosi, 5, Busto Arsizio',
    'FC MADDALENA': 'Piazza S. Stefano, 3, Maddalena',
    'O.S.G.B. BESNATE': 'Via De Gasperi, 5, Jerago con Orago',
    'ORATORIO S.STEFANO': 'Via Bonacalza, 154, Oggiona con Santo Stefano',
    'OSGB MIC GALLARATE/C': 'Via Madonna in Campagna, Gallarate',
    'REAL BUSTO B': 'Via Rodari, 25, Busto Arsizio',
}


In [4]:
calendar_df = pd.DataFrame()
for data in matches:
    data = data.drop(columns=[3, 5, 6, 7, 8], axis=1)
    data = data.rename(columns={0: 'date', 1: 'time', 2: 'home', 4: 'away'})
    data = data.query('home == @my_team or away == @my_team').dropna()
    data['where'] = data['home'].map(locations)
    calendar_df = pd.concat([calendar_df, data], ignore_index=True)
calendar_df.head(100)

,date,time,home,away,where
0,14/10/2023,15:00,ASD SAN MAURIZIO BLU,ASD SAN MAURIZIO NERO,"Via San Pio X, 57, Cassano Magnago"
1,20/10/2023,21:15,ASD SAN MAURIZIO NERO,ORATORIO S.STEFANO,"Via L. Da Vinci, 8, Cassano Magnago"
2,28/10/2023,15:30,DUDE CAFÈ,ASD SAN MAURIZIO NERO,"Via Romagnosi, 5, Busto Arsizio"
3,03/11/2023,21:15,ASD SAN MAURIZIO NERO,OSGB MIC GALLARATE/C,"Via L. Da Vinci, 8, Cassano Magnago"
4,11/11/2023,16:30,FC MADDALENA,ASD SAN MAURIZIO NERO,"Piazza S. Stefano, 3, Maddalena"
5,17/11/2023,21:15,ASD SAN MAURIZIO NERO,CENTRO DELLA GIOVENTÙ,"Via L. Da Vinci, 8, Cassano Magnago"
6,25/11/2023,17:30,O.S.G.B. BESNATE,ASD SAN MAURIZIO NERO,"Via De Gasperi, 5, Jerago con Orago"
7,01/12/2023,21:15,ASD SAN MAURIZIO NERO,ASD ORATORI CARDANO OPEN A,"Via L. Da Vinci, 8, Cassano Magnago"
8,08/12/2023,20:30,CSI NUOVA ABBIATE,ASD SAN MAURIZIO NERO,"Via Oslavia, 17, Tradate"
9,26/01/2024,21:15,ASD SAN MAURIZIO NERO,REAL BUSTO B,"Via L. Da Vinci, 8, Cassano Magnago"


In [6]:
from icalendar import Calendar, Event

calendar = Calendar()

def get_opponent(row):
    if row['home'] == my_team:
        return row['away']
    else:
        return row['home']

def get_datetime(row):
    return pd.to_datetime(row['date'] + ' ' + row['time'], format='%d/%m/%Y %H:%M')

for idx, data in calendar_df.iterrows():
    event = Event()
    event.add('title', get_opponent(data))
    event.add('summary', f"{data['home']} - {data['away']}")
    event.add('location', data['where'])
    event.add('dtstart', get_datetime(data))
    calendar.add_component(event)

with open('my-football-calendar.ics', 'wb') as f:
    f.write(calendar.to_ical())